Chargement des données

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the data
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')

On charge les données de films et de notations à partir des fichiers CSV.

Prétraitement des genres de films

In [17]:
# Nettoyage et prétraitement des genres de films
genres_exploded = movies[['movieId', 'genres']].copy()
genres_exploded['genres'] = genres_exploded['genres'].str.split('|')
genres_exploded = genres_exploded.explode('genres')


On nettoie et prétraite la colonne des genres pour la convertir en une liste de genres distincts pour chaque film.

Encodage des genres de films

In [18]:

# Encodage OneHot des genres


encoder = OneHotEncoder()
genres_encoded = encoder.fit_transform(genres_exploded[['genres']]).toarray()
genres_labels = encoder.get_feature_names_out(['genres'])

# Combiner l'identifiant du film et les genres encodés dans un DataFrame
genres_encoded_df = pd.DataFrame(genres_encoded, columns=genres_labels)
genres_encoded_df['movieId'] = genres_exploded['movieId'].values

# Regroupement par movieId et agrégation de l'encodage du genre
movie_genres_encoded = genres_encoded_df.groupby('movieId').max().reset_index()

# Fusionner avec les films pour s'assurer que l'ordre de l'identifiant du film correspond.s
movies = movies.merge(movie_genres_encoded, on='movieId', how='left')

# Préparer la matrice content_input
content_input_matrix = movies.drop(columns=['movieId', 'title', 'genres']).values

# Crée un mapping de movieId à l'index de la matrice content_input_matrix
movie_id_mapping = {id_: idx for idx, id_ in enumerate(movies['movieId'].values)}

# S'assurer que tout les movieIds dans les notes sont présents dans le movie_id_mapping
valid_movie_ids = set(movie_id_mapping.keys())
ratings = ratings[ratings['movieId'].isin(valid_movie_ids)]
# Normalisation des notes
scaler = MinMaxScaler()
#ratings['rating'] = scaler.fit_transform(ratings[['rating']])
# Preparer les données d'entrainement et de validation
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Mapper les movieId à l'index de la matrice content_input_matrix
train_movie_indices = train_data['movieId'].map(movie_id_mapping).values
test_movie_indices = test_data['movieId'].map(movie_id_mapping).values

X_train = [train_data['userId'].values, train_movie_indices, content_input_matrix[train_movie_indices]]
y_train = train_data['rating'].values

X_val = [test_data['userId'].values, test_movie_indices, content_input_matrix[test_movie_indices]]
y_val = test_data['rating'].values

# Verifier les formes des données
print("Shapes de X_train:", [x.shape for x in X_train])
print("Shapes de X_val:", [x.shape for x in X_val])


Shapes de X_train: [(80668,), (80668,), (80668, 20)]
Shapes de X_val: [(20168,), (20168,), (20168, 20)]


On utilise OneHotEncoder pour convertir les genres en une représentation binaire. Chaque genre est transformé en une colonne distincte dans la DataFrame.

Créationn du modèle

In [19]:
from keras.api.models import Model
from keras.api.layers import LSTM, Conv1D, GlobalMaxPooling1D, Input, Embedding, Flatten, Dense, Concatenate, Dropout, MaxPooling1D, Reshape
from keras.api.optimizers import Adam
from keras.api.callbacks import EarlyStopping

# Préparer l'utilisateur et les films intégrés
num_users = ratings['userId'].max()+1  
num_movies = ratings['movieId'].max()+1  


# Definir le model
def create_model():
    # Définition des inputs
    user_input = Input(shape=(1,), name='user_input')
    movie_input = Input(shape=(1,), name='movie_input')
    content_input = Input(shape=(content_input_matrix.shape[1], 1), name='content_input')

    # Embeddings

    user_embedding = Embedding(input_dim=num_users, output_dim=50, name='user_embedding')(user_input)
    movie_embedding = Embedding(input_dim=num_movies, output_dim=50, name='movie_embedding')(movie_input)

    # Flatten embeddings

    user_vector = Flatten()(user_embedding)
    movie_vector = Flatten()(movie_embedding)
    # Embeddings concatener
    user_movie_concatenated = Concatenate()([user_vector, movie_vector])
    # Réseau dense pour les embeddings
    dense_layer1 = Dense(128, activation='relu')(user_movie_concatenated)
    dropout_layer1 = Dropout(0.5)(dense_layer1)
    dense_layer2 = Dense(64, activation='relu')(dropout_layer1)
    # Réseau dense pour le contenu
    content_cnn = Conv1D(64, kernel_size=3, activation='relu')(content_input)
    content_cnn = MaxPooling1D(pool_size=2)(content_cnn)
    content_cnn = Flatten()(content_cnn)
    dense_layer3 = Dense(128, activation='relu')(content_cnn)
    dropout_layer2 = Dropout(0.5)(dense_layer3)
    dense_layer4 = Dense(64, activation='relu')(dropout_layer2)
    # Concatenation des deux réseaux
    final_concatenated = Concatenate()([dense_layer2, dense_layer4])
    output = Dense(1)(final_concatenated)

    model = Model(inputs=[user_input, movie_input, content_input], outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

model = create_model()
print(model.summary())

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ content_input       │ (None, 20, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 50)     │     30,550 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_embedding     │ (None, 1, 50)     │  9,680,500 │ movie_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 18, 64)    │        256 │ content_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 50)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 50)        │          0 │ movie_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 9, 64)     │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 100)       │          0 │ flatten_3[0][0],  │
│ (Concatenate)       │                   │            │ flatten_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 576)       │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     12,928 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │     73,856 │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │      8,256 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      8,256 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 128)       │          0 │ dense_6[0][0],    │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │        129 │ concatenate_3[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,814,731 (37.44 MB)

 Trainable params: 9,814,731 (37.44 MB)

 Non-trainable params: 0 (0.00 B)

None


On a un modèle de réseau de neurones hybride qui combine des embeddings utilisateurs/films et des informations de contenu (genres). Le modèle est compilé avec une fonction de perte mean_squared_error et l'optimiseur Adam.

Entrainement du modèle

In [20]:
# Entrainer le modèle 
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

Epoch 1/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 52s 40ms/step - loss: 1.5814 - mae: 0.9328 - val_loss: 0.8653 - val_mae: 0.7417
Epoch 2/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - loss: 0.7541 - mae: 0.6725 - val_loss: 0.8057 - val_mae: 0.7060
Epoch 3/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 51s 40ms/step - loss: 0.6980 - mae: 0.6415 - val_loss: 0.7800 - val_mae: 0.6894
Epoch 4/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 51s 40ms/step - loss: 0.6556 - mae: 0.6189 - val_loss: 0.7626 - val_mae: 0.6697
Epoch 5/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 51s 40ms/step - loss: 0.6258 - mae: 0.6033 - val_loss: 0.7629 - val_mae: 0.6663
Epoch 6/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 52s 41ms/step - loss: 0.5832 - mae: 0.5813 - val_loss: 0.7643 - val_mae: 0.6667
Epoch 7/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 49s 39ms/step - loss: 0.5620 - mae: 0.5688 - val_loss: 0.7806 - val_mae: 0.6711


On entraine le modèle avec les données de formation et de validation sur 10 époques avec une taille de batch de 64.

In [23]:
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score, accuracy_score
import numpy as np

# Prédire les notes
y_pred = model.predict(X_val)

# Calcul des métriques
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
precision = precision_score(y_val >= 4, y_pred >= 4, average='weighted', zero_division=0)
recall = recall_score(y_val >= 4, y_pred >= 4, average='weighted', zero_division=0)
f1 = f1_score(y_val >= 4, y_pred >= 4, average='weighted', zero_division=0)
accuracy = accuracy_score(y_val >= 4, y_pred >= 4)
# Afficher les métriques
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Accuracy: {accuracy}')


631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Root Mean Squared Error (RMSE): 0.8732662315956033
Precision: 0.7031131855613657
Recall: 0.6446846489488298
F1 Score: 0.6094475466123699
Accuracy: 0.6446846489488298


Exemple de la fonction de recommandation appliqué

In [26]:

# Defintion de notre fonction de recommandation
def recommend_movies(user_id, num_recommendations=5):
    # recupérer tous les movieIds
    all_movie_ids = movies['movieId'].values
    
    # preparer un tableau d'identifiant d'utilisateur
    user_array = np.array([user_id] * len(all_movie_ids))
    
    # Preparer les données d'entrée pour le contenu
    content_input_all_movies = content_input_matrix
    
    # Prédire les notes pour tous les films
    predictions = model.predict([user_array, all_movie_ids, content_input_all_movies])
    
    # Creer un DataFrame avec les movieIds et les notes prédites
    pred_df = pd.DataFrame({'movieId': all_movie_ids, 'predicted_rating': predictions.flatten()})
    
    # Fusionner avec les films
    recommendations = pred_df.merge(movies, on='movieId')
    
    # Trier les recommandations par note prédite et obtenir les 5 premières
    top_recommendations = recommendations.sort_values(by='predicted_rating', ascending=False).head(num_recommendations)
    
    return top_recommendations[['movieId', 'title', 'predicted_rating']]

# Exaemple : Recommander des films pour l'utilisateur ID 
user_id = 75
recommendations = recommend_movies(user_id, num_recommendations=5)
print("Top 5 des films recommandés pour l'utilisateur ID",user_id,":")
print(recommendations)


305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Top 5 des films recommandés pour l'utilisateur ID 75 :
      movieId                                title  predicted_rating
44         48                    Pocahontas (1995)          4.172474
2620     3505                    No Way Out (1987)          4.159114
517       602        Great Day in Harlem, A (1994)          4.139214
1192     1589                      Cop Land (1997)          4.088707
1240     1649  Fast, Cheap & Out of Control (1997)          4.060501
